# Resonant innelastic X-ray scattering (RIXS) 

# Import the required packages and set up logging

In [ ]:
%matplotlib ipympl

import sys
import logging

import numpy as np
import scipy
import matplotlib.pyplot as plt

from daxs.sources import Hdf5Source
from daxs.measurements import Rixs

logging.basicConfig(level=logging.INFO, stream=sys.stdout)
logging.getLogger("daxs").setLevel(logging.DEBUG)

## Load the experimental data

In [ ]:
filename = "experimental_data/ihch1527/id26/Sami_Ni_MOF_RIXS_cc/Sami_Ni_MOF_RIXS_cc_Ka1/Sami_Ni_MOF_RIXS_cc_Ka1.h5"
included_scans = "3-83"
data_mappings = {
    "x": ".1/measurement/hdh_energy", 
    "y": ".1/instrument/positioners/xes_en",
    "signal": ".1/measurement/det_dtc_apd", 
    "monitor": ".1/measurement/I02",
}
source = Hdf5Source(filename=filename, included_scans=included_scans, data_mappings=data_mappings)

measurement = Rixs(source)

## A note on plotting 2D data

In [ ]:
x, y, signal = measurement.x, measurement.y, measurement.signal

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(7.8, 4.5), sharey=True)

# Plot using tricontourf.
levels = 101
ax1.tricontourf(x, x - y, signal, levels=levels)
ax1.set_xlabel("Incident energy (keV)")
ax1.set_ylabel("Emission energy (keV)")
ax1.set_title("Using tricontourf")

# Plot using imshow. The data needs to be reshaped.
nscans = 81
npoints_per_scan = 66
signal_reshaped = signal.reshape((nscans, npoints_per_scan))

# https://matplotlib.org/stable/gallery/images_contours_and_fields/interpolation_methods.html
interpolation = "bilinear"
extent = [x.min(), x.max(), (x - y).min(), (x - y).max()]
ax2.imshow(
    signal_reshaped,
    origin="lower",
    extent=extent,
    aspect="auto",
    interpolation=interpolation,
)
ax2.set_xlabel("Incident energy (keV)")
ax2.set_title("Using imshow")

plt.tight_layout()


## Emission energy vs energy transfer plots

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4.5))

# Plot the plane with the emission energy as Y-axis.
ax1.tricontourf(x, x - y, signal, levels=levels)
ax1.set_xlabel("Incident energy (keV)")
ax1.set_ylabel("Emission energy (keV)")

# Plot the plane with the energy transfer as Y-axis.
ax2.tricontourf(x, y, signal, levels=levels)
ax2.set_xlabel("Incident energy (keV)")
ax2.set_ylabel("Energy transfer (keV)")

plt.tight_layout()

## Apply the concentration correction

In [ ]:
measurement.reset()

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4.5), sharex=True, sharey=True)

# Plot the uncorrected plane.
ax1.tricontourf(measurement.x, measurement.y, measurement.signal, levels=levels)
ax1.set_xlabel("Incident energy (keV)")
ax1.set_ylabel("Emission energy (keV)")

measurement.concentration_correction(84)

# Plot the plane concentration-corrected plane.
ax2.tricontourf(measurement.x, measurement.y, measurement.signal, levels=levels)
ax2.set_xlabel("Incident energy (keV)")
ax2.set_ylabel("Energy transfer (keV)")

plt.tight_layout()

## Cut the plane

In [ ]:
measurement.cut("CEE", energies=[7.4780, 7.4790, 7.4794])
measurement.cut("CIE", energies=[8.332])
measurement.cut("CET", [0.8531])

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8.5, 4.5))

ax1.tricontourf(measurement.x, measurement.y, measurement.signal, levels=levels)

for label, (x, y, signal) in measurement.cuts.items():
    if "CIE" in label:
        continue
        
    ax1.plot(x, y)

    ids = np.argsort(x)
    x = x[ids]
    y = y[ids]
    signal = signal[ids]

    ax2.plot(x, signal, label=label)

ax1.set_xlabel("Incident energy (keV)")
ax1.set_ylabel("Energy transfer (keV)")

ax2.set_xlabel("Incident energy (keV)")
ax2.set_ylabel("Intensity (arb. units)")

ax2.legend() 
plt.tight_layout()

## Interpolate the plane

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4.5), sharex=True, sharey=True)

measurement.reset()

# Plot the plane original plane.
ax1.tricontourf(measurement.x, measurement.y, measurement.signal, levels=levels)
ax1.set_xlabel("Incident energy (keV)")
ax1.set_ylabel("Energy transfer (keV)")
ax1.set_title(f"Original signal: {len(measurement.signal)} points")

npoints = 51
shift = 5e-5
x, y, signal = measurement.x, measurement.y, measurement.signal
xi = np.linspace(x.min() + shift, x.max() - shift, npoints)
yi = np.linspace(y.min() + shift, y.max() - shift, npoints)
measurement.interpolate(xi, yi)

# Plot the plane the interpolated plane.
ax2.tricontourf(measurement.x, measurement.y, measurement.signal, levels=levels)
ax2.set_xlabel("Incident energy (keV)")
ax2.set_title(f"Interpolated signal: {len(measurement.signal)} points")

plt.tight_layout()